In [ ]:
import numpy as np

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
import cv2 as cv

In [ ]:
from sklearn import linear_model

In [ ]:
import warnings

In [ ]:
plt.style.use('seaborn')

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
def plot_image(image, i=0):

In [ ]:
    l = [cv.COLOR_BGR2RGB, cv.COLOR_BGR2GRAY, cv.COLOR_RGB2GRAY,]

In [ ]:
    plt.imshow(cv.cvtColor(image, l[i]))

In [ ]:
    plt.show(block=True)

In [ ]:
def display_image(image):

In [ ]:
    cv.imshow('image', image)

In [ ]:
    cv.waitKey(0)

In [ ]:
    cv.destroyAllWindows()

In [ ]:
def plot_graph(image, i=0):

In [ ]:
    plt.figure(figsize=(15, 8))

In [ ]:
    sns.scatterplot(np.arange(len(image.sum(i))), image.sum(i), label='a', alpha=0.6)

In [ ]:
def MinMaxScalar(x, a=-1, b=1):

In [ ]:
    N = (x - x.min()) * (b - a)

In [ ]:
    D = (x.max() - x.min())

In [ ]:
    return (N / D) + a